In [1]:
# Set up
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import pandas as pd
import seaborn as sns
import statsmodels.formula.api as smf
import statsmodels.api as sm
import matplotlib as mpl
import patsy as pt
    
import wooldridge as woo

###  Use data in DISCRIM from wooldridge

In [2]:
discrim = woo.data('DISCRIM')
discrim.head()

,psoda,pfries,pentree,wagest,nmgrs,nregs,hrsopen,emp,psoda2,pfries2,...,county,lpsoda,lpfries,lhseval,lincome,ldensity,NJ,BK,KFC,RR
0,1.12,1.06,1.02,4.25,3.0,5.0,16.0,27.5,1.11,1.11,...,18,0.113329,0.058269,11.906993,10.704008,8.301521,1,0,0,1
1,1.06,0.91,0.95,4.75,3.0,3.0,16.5,21.5,1.05,0.89,...,18,0.058269,-0.094311,11.906993,10.704008,8.301521,1,1,0,0
2,1.06,0.91,0.98,4.25,3.0,5.0,18.0,30.0,1.05,0.94,...,12,0.058269,-0.094311,12.038836,10.625319,9.341369,1,1,0,0
3,1.12,1.02,1.06,5.00,4.0,5.0,16.0,27.5,1.15,1.05,...,10,0.113329,0.019803,12.052921,10.827071,9.029418,1,0,0,1
4,1.12,NaN,0.49,5.00,3.0,3.0,16.0,5.0,1.04,1.01,...,10,0.113329,NaN,12.425610,11.188399,6.579251,1,1,0,0


(a) Estimate the regression model:
log(psoda) = β0 + β1prpblck + β2log(income) + β3prppov + u
and report the results in the usual form. Is β1 statistically different from zero at the 5% level against
a two-sided alternative? What about at the 1% level?


In [3]:
ols_mod = smf.ols(formula = 'lpsoda ~ prpblck + lincome + prppov',data=discrim)
ols_fit = ols_mod.fit()
ols_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 lpsoda   R-squared:                       0.087
Model:                            OLS   Adj. R-squared:                  0.080
Method:                 Least Squares   F-statistic:                     12.60
Date:                Mon, 14 Nov 2022   Prob (F-statistic):           6.92e-08
Time:                        18:02:35   Log-Likelihood:                 439.04
No. Observations:                 401   AIC:                            -870.1
Df Residuals:                     397   BIC:                            -854.1
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.4633      0.294     -4.982      0.000      -2.041      -0.886
prpblck        0.0728      0.031      2.373      0.018       0.013       0.133
lincome        0.1370      0.027      5.119      0.000       0.084       0.190
prppov         0.3804      0.133      2.864      0.004       0.119       0.641
==============================================================================
Omnibus:                       12.002   Durbin-Watson:                   1.727
Prob(Omnibus):                  0.002   Jarque-Bera (JB):               24.056
Skew:                          -0.014   Prob(JB):                     5.97e-06
Kurtosis:                       4.200   Cond. No.                         834.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""


p-value for prpblck is 0.018, smaller than 0.05 but bigger than 0.01. This suggest that β1 is statistically different from zero at the 5% level against a two-sided alternative but not with 1% level.

(b)What is the correlation between log(income) and prppov? VIF? Is each variable statistically significant in any case? Report the two-sided p-values.


In [4]:
## correlation
print("the correlation between log(income) and prppov is", discrim['lincome'].corr(discrim['prppov']))

the correlation between log(income) and prppov is -0.8384669885470059


In [5]:
import patsy as pt
import statsmodels.stats.outliers_influence as smo

# Get the design matrix
y, X = pt.dmatrices('lpsoda ~ prpblck + lincome + prppov', data = discrim,
                   return_type = 'dataframe')

X.head()

,Intercept,prpblck,lincome,prppov
0,1.0,0.171154,10.704008,0.036579
1,1.0,0.171154,10.704008,0.036579
2,1.0,0.047360,10.625319,0.087907
3,1.0,0.052839,10.827071,0.059123
4,1.0,0.034480,11.188399,0.025415


In [6]:
#  Pull the number of regressors (+ intercept)
k = X.shape[1]

# create an empty matrix to store results
VIF = np.empty(k)

# Loop for each regressor (+ intercept)
for i in range(k):
    
    # calculate the VIF for each
    VIF[i] = smo.variance_inflation_factor(X.values, i)

print('VIF:', VIF)

VIF: [5.22495429e+03 1.92215973e+00 3.51872158e+00 4.91522152e+00]


vif of prppov is larger than 4, which indicates multicollinearity.

In [7]:
##p-values
print(ols_fit.pvalues)

Intercept    9.400129e-07
prpblck      1.809769e-02
lincome      4.802010e-07
prppov       4.400338e-03
dtype: float64


all variables are statistically significant according to the p-values

(c)  To the regression in part (a), add the variable log(hseval). Interpret its coefficient and
report the two-sided p-value for H0 : βlog(hseval) = 0.

In [8]:
ols_mod_c = smf.ols('lpsoda ~ prpblck + lincome + prppov + lhseval',data=discrim)
ols_fit_c = ols_mod_c.fit()
ols_fit_c.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 lpsoda   R-squared:                       0.184
Model:                            OLS   Adj. R-squared:                  0.176
Method:                 Least Squares   F-statistic:                     22.31
Date:                Mon, 14 Nov 2022   Prob (F-statistic):           1.24e-16
Time:                        18:02:36   Log-Likelihood:                 461.55
No. Observations:                 401   AIC:                            -913.1
Df Residuals:                     396   BIC:                            -893.1
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.8415      0.292     -2.878      0.004      -1.416      -0.267
prpblck        0.0976      0.029      3.334      0.001       0.040       0.155
lincome       -0.0530      0.038     -1.412      0.159      -0.127       0.021
prppov         0.0521      0.134      0.388      0.699      -0.212       0.317
lhseval        0.1213      0.018      6.860      0.000       0.087       0.156
==============================================================================
Omnibus:                       16.452   Durbin-Watson:                   1.973
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               40.077
Skew:                          -0.016   Prob(JB):                     1.98e-09
Kurtosis:                       4.548   Cond. No.                     1.31e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.31e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

The p-value of lhseval is 0.000, which suggests result is statistically significant.
The coefficient of lhseval is 0.1213, suggesing that holding other variables same, 1 percent change in median housing value will lead to 12.13% change in price of medium soda. 

(d)In the regression in part (c), what happens to the individual statistical significance of
log(income) and prppov? Are these variables jointly significant? (Compute a p-value.) What do
you make of your answers?

In [9]:
hypotheses = '(prppov=0), (lincome=0)'
f_test = ols_fit_c.f_test(hypotheses)
print(f_test)

<F test: F=3.5226849440225116, p=0.030448605108275816, df_denom=396, df_num=2>



The p-value of prpov and log(income) is not significant anymore.  
The p-value of f-test is statistically significant, suggesting that median family income and proportion in poverty are jointly significant.

(e) Given the results of the previous regressions, which one would you report as most reliable
in determining whether the racial makeup of a zip code influences local fast-food prices?

 
I will choose the model from question c. It has a bigger F-value, suggesting it is more useful in predicting local fast-food prices. Although log(income) and proportion in poverty are not significant separately, they are jointly significant. This means that they should still be kept in the model.

###  Use the VOTE1 dataset from the Wooldridge python module


(a) Estimate the following model:
voteA = β0 + β1prtystrA + β2log(expendA) + β3democA + β4log(expendB) + u
Obtain the OLS residuals, ˆui
, and regress these on all of the independent variables. Explain why
you obtain R2 = 0.


In [10]:
vote1 = woo.data('VOTE1')
ols_mod_v = smf.ols('voteA ~ prtystrA + np.log(expendA) + democA + np.log(expendB)',data=vote1)
result_v = ols_mod_v.fit()
residuals = result_v.resid
ols_mod_v2 = smf.ols('residuals ~ prtystrA + np.log(expendA) + democA + np.log(expendB)',data=vote1)
result_v2 = ols_mod_v2.fit()
result_v2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              residuals   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.024
Method:                 Least Squares   F-statistic:                     0.000
Date:                Mon, 14 Nov 2022   Prob (F-statistic):               1.00
Time:                        18:02:36   Log-Likelihood:                -593.20
No. Observations:                 173   AIC:                             1196.
Df Residuals:                     168   BIC:                             1212.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept       -9.334e-14      4.736  -1.97e-14      1.000      -9.350       9.350
prtystrA         8.674e-17      0.071   1.22e-15      1.000      -0.141       0.141
np.log(expendA)   5.01e-15      0.392   1.28e-14      1.000      -0.774       0.774
democA           7.772e-15      1.407   5.53e-15      1.000      -2.777       2.777
np.log(expendB)  6.772e-15      0.397    1.7e-14      1.000      -0.785       0.785
==============================================================================
Omnibus:                        6.304   Durbin-Watson:                   1.525
Prob(Omnibus):                  0.043   Jarque-Bera (JB):                6.030
Skew:                           0.448   Prob(JB):                       0.0491
Kurtosis:                       3.182   Cond. No.                         429.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""


R^2=SSE/SST=Σ(ŷ-ȳ)^2/SST. Since y in the new regression is the residuals(sums up to 0), SSE = 0, leading R^2 zero as well. 

(b)compute the Breusch-Pagan test for heteroskedasticity. Use the F statistic version
and report the p-value.

In [11]:
y, X = pt.dmatrices('voteA ~ prtystrA + np.log(expendA) + democA + np.log(expendB)', vote1,
                   return_type = 'dataframe')

# Order is Lm Test statistic, LM P-value, F-stat, F-Pvalue
sm.stats.diagnostic.het_breuschpagan(residuals, X)

(9.093355957811195,
 0.05880791685694257,
 2.3301126837162602,
 0.058057514088554106)


p-value = 0.058.  We can not reject the null H0: variance = constant. Heteroskedasticity is not present.

(c) Compute the special case of the White test for heteroskedasticity, again using the F
statistic form. How strong is the evidence for heteroskedasticity now?

In [12]:
# Order is Lm Test statistic, LM P-value, F-stat, F-Pvalue
sm.stats.diagnostic.het_white(residuals, X)

(31.101516025646685,
 0.003258261608789093,
 2.68075778248278,
 0.0019733250042554325)


p-value = 0.002. Null H0: variance = constant is rejected. Heteroskedasticity is present now at 0.5% significance level. This is because the White Test solves the problem of BP test by using existing predictors from the original model.

(d)  Estimate two more models. For the first use robust standard errors (HC0) and for the
second use the FGLS procedure. Note any changes in your standard errors, significance level, and
coefficients.

In [13]:
## reg1 with HC0
reg1 = smf.ols('voteA ~ prtystrA + np.log(expendA) + democA + np.log(expendB) + district', vote1)
reg1.fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  voteA   R-squared:                       0.804
Model:                            OLS   Adj. R-squared:                  0.798
Method:                 Least Squares   F-statistic:                     136.6
Date:                Mon, 14 Nov 2022   Prob (F-statistic):           4.19e-57
Time:                        18:02:36   Log-Likelihood:                -592.17
No. Observations:                 173   AIC:                             1196.
Df Residuals:                     167   BIC:                             1215.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          38.1260      4.734      8.054      0.000      28.781      47.471
prtystrA            0.2604      0.071      3.650      0.000       0.120       0.401
np.log(expendA)     5.7677      0.391     14.760      0.000       4.996       6.539
democA              4.0180      1.411      2.847      0.005       1.232       6.805
np.log(expendB)    -6.2626      0.397    -15.788      0.000      -7.046      -5.479
district           -0.0938      0.066     -1.412      0.160      -0.225       0.037
==============================================================================
Omnibus:                        5.609   Durbin-Watson:                   1.536
Prob(Omnibus):                  0.061   Jarque-Bera (JB):                5.318
Skew:                           0.423   Prob(JB):                       0.0700
Kurtosis:                       3.153   Cond. No.                         436.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [14]:
robust_reg1 = smf.ols('voteA ~ prtystrA + np.log(expendA) + democA + np.log(expendB) + district', vote1).fit(cov_type = 'HC0')
robust_reg1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  voteA   R-squared:                       0.804
Model:                            OLS   Adj. R-squared:                  0.798
Method:                 Least Squares   F-statistic:                     136.1
Date:                Mon, 14 Nov 2022   Prob (F-statistic):           5.36e-57
Time:                        18:02:36   Log-Likelihood:                -592.17
No. Observations:                 173   AIC:                             1196.
Df Residuals:                     167   BIC:                             1215.
Df Model:                           5                                         
Covariance Type:                  HC0                                         
===================================================================================
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          38.1260      4.354      8.757      0.000      29.593      46.659
prtystrA            0.2604      0.064      4.087      0.000       0.135       0.385
np.log(expendA)     5.7677      0.524     11.003      0.000       4.740       6.795
democA              4.0180      1.430      2.810      0.005       1.216       6.820
np.log(expendB)    -6.2626      0.351    -17.840      0.000      -6.951      -5.575
district           -0.0938      0.059     -1.592      0.111      -0.209       0.022
==============================================================================
Omnibus:                        5.609   Durbin-Watson:                   1.536
Prob(Omnibus):                  0.061   Jarque-Bera (JB):                5.318
Skew:                           0.423   Prob(JB):                       0.0700
Kurtosis:                       3.153   Cond. No.                         436.
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
"""


The parameters doesn't change much, suggesting model robustness.

In [15]:
##reg2 with FGLS
reg2 = smf.ols('voteA ~ np.log(expendA) +  np.log(expendB) + district', vote1)
reg2.fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  voteA   R-squared:                       0.786
Model:                            OLS   Adj. R-squared:                  0.783
Method:                 Least Squares   F-statistic:                     207.4
Date:                Mon, 14 Nov 2022   Prob (F-statistic):           2.06e-56
Time:                        18:02:36   Log-Likelihood:                -599.38
No. Observations:                 173   AIC:                             1207.
Df Residuals:                     169   BIC:                             1219.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          52.7736      2.849     18.523      0.000      47.149      58.398
np.log(expendA)     6.3496      0.373     17.034      0.000       5.614       7.086
np.log(expendB)    -6.7924      0.382    -17.799      0.000      -7.546      -6.039
district           -0.0676      0.068     -0.988      0.325      -0.203       0.067
==============================================================================
Omnibus:                        6.780   Durbin-Watson:                   1.671
Prob(Omnibus):                  0.034   Jarque-Bera (JB):                6.431
Skew:                           0.429   Prob(JB):                       0.0401
Kurtosis:                       3.395   Cond. No.                         65.5
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [16]:
vote1["ehat"] = reg2.fit().resid**2

# estimate weights
w_est = smf.ols('np.log(ehat) ~ np.log(expendA) +  np.log(expendB) + district', data = vote1).fit()

vari = np.exp(w_est.fittedvalues) #estimated variances
w = 1/vari**2

fgls =smf.wls('voteA ~ np.log(expendA) +  np.log(expendB) + district', vote1, weights = w).fit()

fgls.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:                  voteA   R-squared:                       0.709
Model:                            WLS   Adj. R-squared:                  0.704
Method:                 Least Squares   F-statistic:                     137.6
Date:                Mon, 14 Nov 2022   Prob (F-statistic):           3.81e-45
Time:                        18:02:36   Log-Likelihood:                -607.56
No. Observations:                 173   AIC:                             1223.
Df Residuals:                     169   BIC:                             1236.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          51.8440      3.484     14.880      0.000      44.966      58.722
np.log(expendA)     7.4229      0.489     15.189      0.000       6.458       8.388
np.log(expendB)    -7.7087      0.459    -16.790      0.000      -8.615      -6.802
district           -0.0605      0.059     -1.031      0.304      -0.176       0.055
==============================================================================
Omnibus:                        3.983   Durbin-Watson:                   1.710
Prob(Omnibus):                  0.136   Jarque-Bera (JB):                4.923
Skew:                          -0.050   Prob(JB):                       0.0853
Kurtosis:                       3.820   Cond. No.                         96.6
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""


The coefficients, standard errors and p values stays the same